In [2]:
import pandas as pd 
import numpy as np 
import geopandas as gpd
import rasterio
import pystac_client

In [4]:

def extract_gretter_fire_per_year(year=2022, tile='22LHK'): 
    service = pystac_client.Client.open('https://data.inpe.br/bdc/stac/v1/')
    im= pd.read_json(f'early_burned_area_{year}_{tile}_30.json')
    after=im[im['ba_detect'] == 1].sort_values(by='firespots_that', ascending=False).iloc[0]['item_after']
    before=im[im['ba_detect'] == 1].sort_values(by='firespots_that', ascending=False).iloc[0]['item_before']
    item_search_before = service.search(ids=[f'{before}'],
                                collections=['S2_L2A-1'],
                                )
    item_search_before.matched()
    item_search_after = service.search(ids=[f'{after}'],
                                collections=['S2_L2A-1'],
                                )
    item_search_after.matched()

    for item in item_search_before.items():
        assets_before = item.assets
    assets_before

    for item in item_search_after.items():
        assets_after = item.assets
    assets_after

    swir_asset_after = assets_after['B12']
    swir_asset_after.href
    swir1_asset_after = assets_after['B11']
    swir1_asset_after.href
    scl_asset_after = assets_after['SCL']
    scl_asset_after.href
    swir_asset_before = assets_before['B12']
    swir_asset_after.href
    swir1_asset_before = assets_before['B11']
    swir1_asset_before.href
    scl_asset_before = assets_after['SCL']
    scl_asset_before.href

    # Abrindo os rasters
    with rasterio.open(swir1_asset_after.href) as src_b11, rasterio.open(swir_asset_after.href) as src_b12:
        # Lendo as bandas como arrays
        b11 = src_b11.read(1).astype('float32')
        b12 = src_b12.read(1).astype('float32')

        # Evitando divisão por zero
        soma = b11 + b12
        soma[soma == 0] = np.nan  # Define como NaN onde a soma for zero

        # Calculando o NBR
        nbr = (b11 - b12) / soma

        # Configuração dos metadados para o novo raster
        meta = src_b11.meta.copy()
        meta.update(dtype=rasterio.float32, count=1)
        with rasterio.open(f'after_{year}_{tile}.tif', 'w', **meta) as dst:
            dst.write(b11, 1)

        # Salvando o raster do NBR
        with rasterio.open(f'after_fire_{year}_{tile}.tif', 'w', **meta) as dst:
            dst.write(nbr, 1)

    print(f"NBR salvo em: after_fire_2022.tif")

    with rasterio.open(swir1_asset_before.href) as src_b11, rasterio.open(swir_asset_before.href) as src_b12:
        # Lendo as bandas como arrays
        b11 = src_b11.read(1).astype('float32')
        b12 = src_b12.read(1).astype('float32')
        
        # Evitando divisão por zero
        soma = b11 + b12
        soma[soma == 0] = np.nan  # Define como NaN onde a soma for zero

        # Calculando o NBR
        nbr = (b11 - b12) / soma

        # Configuração dos metadados para o novo raster
        meta = src_b11.meta.copy()
        meta.update(dtype=rasterio.float32, count=1)
        with rasterio.open(f'before_{year}_{tile}.tif', 'w', **meta) as dst:
            dst.write(b11, 1)

        # Salvando o raster do NBR
        with rasterio.open(f'before_fire_{year}_{tile}.tif', 'w', **meta) as dst:
            dst.write(nbr, 1)
    create_rgb_image(year, tile)
    print(f"NBR salvo em: after.tif")

def create_rgb_image(year=2022, tile='22LHK'):
    # Abrindo os rasters antes e depois do incêndio
    with rasterio.open(f'before_{year}_{tile}.tif') as src_r, \
         rasterio.open(f'before_fire_{year}_{tile}.tif') as src_g, \
         rasterio.open(f'after_fire_{year}_{tile}.tif') as src_b:

        # Lendo os dados das imagens
        red = src_r.read(1).astype(np.float32)  # before
        green = src_g.read(1).astype(np.float32)  # before_fire
        blue = src_b.read(1).astype(np.float32)  # after_fire

        # Criando um array empilhado com os três canais
        rgb = np.stack([red, green, blue], axis=0)

        # Configuração dos metadados para a nova imagem
        meta = src_r.meta.copy()
        meta.update(dtype=rasterio.float32, count=3)

        # Salvando a imagem RGB
        with rasterio.open(f'rgb_fire_{year}_{tile}.tif', 'w', **meta) as dst:
            dst.write(rgb)

    print(f"Imagem RGB salva em: rgb_fire_{year}_{tile}.tif")

if __name__=='__main__':
    for year in range(3):
        year = 2022 + year
        print(year)
        extract_gretter_fire_per_year(year,'22LHL')
    

2022
NBR salvo em: after_fire_2022.tif
Imagem RGB salva em: rgb_fire_2022_22LHL.tif
NBR salvo em: after.tif
2023
NBR salvo em: after_fire_2022.tif
Imagem RGB salva em: rgb_fire_2023_22LHL.tif
NBR salvo em: after.tif
2024
NBR salvo em: after_fire_2022.tif
Imagem RGB salva em: rgb_fire_2024_22LHL.tif
NBR salvo em: after.tif


In [6]:
import rasterio
import numpy as np

with rasterio.open('before_2023_22LHL.tif') as sr1c, \
     rasterio.open('after_2023_22LHL.tif') as sr2c:
    
    # Ler os dados das bandas
    before = sr1c.read(1).astype(np.float32)
    after = sr2c.read(1).astype(np.float32)
    
    # Calcular a diferença e aplicar sqrt em valores absolutos
    dmask = np.sqrt(np.abs(before - after))
    
    # Copiar metadados e atualizar tipo de dado
    meta = sr1c.meta.copy()
    meta.update(dtype=rasterio.float32, count=1)
    
    # Salvar o resultado
    with rasterio.open('dnbr.tif', 'w', **meta) as dst:
        dst.write(dmask[np.newaxis, :, :])  # Adiciona dimensão da banda


In [ ]:
for year in range(3):
    year = 2022 + year
    extract_gretter_fire_per_year(year,'22LHK')
    extract_gretter_fire_per_year(year,'22LHL')
    extract_gretter_fire_per_year(year,'23LKE')
    extract_gretter_fire_per_year(year,'23LKF')